In [ ]:
import qtune.basic_dqd
from qtune.experiment import Measurement

import numpy as np
import pandas as pd
import qtune.evaluator
import qtune.parameter_tuner
import qtune.solver
import qtune.gradient
import qtune.kalman_gradient
import qtune.autotuner
from qtune import gui
import qtune.history

"""
This tutorial explains in detailed step how the qtune package is used to set up an automated fine-tuner. 

The tutorial assumes that you have read the readme, which gives you a general overview of the classes.

The first step
is the connection to the experiment, which depends on your implementation of a specific Experiment class for the 
interface to voltages and an Evaluator subclass for each parameter. 
In this tutorial, these classes are substituted by simple simulations.
"""

'\nThis tutorial explains in detailed step how the qtune package is used to set up an automated fine-tuner. \n\nThe tutorial assumes that you have read the readme, which gives you a general overview of the classes.\n\nThe first step\nis the connection to the experiment, which depends on your implementation of a specific Experiment class for the \ninterface to voltages and an Evaluator subclass for each parameter. \nIn this tutorial, these classes are substituted by simple simulations.\n'

In [ ]:
"""
The important information is the use of the Measurement class. The Simulator will be replaced by an actual measurement.
"""
singlet_reload_simulation = qtune.basic_dqd.Simulator(qtune.basic_dqd.load_simulation, gate1="T", gate2="SA")
default_reload_scan = Measurement("load_scan")
"""
The Simulator class simulates the dependence of the singlet reload time on two gate forming a tunnel barrier, which are 
chosen to be the voltages on the gates T and SA. The simulating function is called "load_simulation" The corresponding 
scan has the name "load_scan".  
"""

'\nThe Simulator class simulates the dependence of the singlet reload time on two gate forming a tunnel barrier, which are \nchosen to be the voltages on the gates T and SA. The simulating function is called "load_simulation" The corresponding \nscan has the name "load_scan".  \n'

In [ ]:
"""
The simulator for the inter-dot tunnel coupling by the transition broadening. The scan sweeps the detuning. A 
Measurement can be given any information keyword argument. They are all saved in the options dictionary.
"""
detune_sim = qtune.basic_dqd.Simulator(qtune.basic_dqd.detune_simulation, central_upper_gate="T",
                                       central_lower_gate="N", left_gate="SB", right_gate="SA")
default_detune_scan = Measurement('detune_scan', center=0., range=2e-3, N_points=100, ramptime=.02, N_average=10,
                                  AWGorDecaDAC='AWG')

In [ ]:
"""
The one dimensional scan of a sensing dot.
"""
ss1d_sim = qtune.basic_dqd.Simulator(qtune.basic_dqd.ss1d_simulation)
sensing_dot_measurement = Measurement('line_scan', center=0., range=4e-3, gate="SDB2", N_points=1280, ramptime=.0005,
                                      N_average=1, AWGorDecaDAC='DecaDAC')
"""
The two dimensional scan of a sensing dot.
"""
ss2d_sim = qtune.basic_dqd.Simulator(qtune.basic_dqd.ss2d_simulation, gate1="SDB1", gate2='SDB2')
ss2d_measurement = Measurement("ss2d", center=[0, 0], gate1='SDB1', gate2='SDB2', range=15e-3, n_lines=20, n_points=104)

In [ ]:
"""
Transitions in the charge diagram.
"""
rfa_trans_sim = qtune.basic_dqd.Simulator(qtune.basic_dqd.transition_simulation, gate_lead="SA", gate_opposite="BA")
rfa_line_scan = Measurement('line_scan', center=0., range=4e-3, gate='RFA', N_points=1280, ramptime=.0005,
                            N_average=3, AWGorDecaDAC='DecaDAC')


rfb_trans_sim = qtune.basic_dqd.Simulator(qtune.basic_dqd.transition_simulation, gate_lead="SB", gate_opposite="BB")
rfb_line_scan = Measurement('line_scan', center=0., range=4e-3, gate='RFB', N_points=1280, ramptime=.0005,
                            N_average=3, AWGorDecaDAC='DecaDAC')

In [ ]:
"""
The initialisation of the simulated experiment. This will be replaced by the actual experiment.
"""
initial_voltages = pd.Series({"SA": -3., "T": -1., "SDB1": 0., "SDB2": 0., "SB": -2., "N": 0., "BA": -4., "BB": -2.,
                              "RFA": 0., "RFB": 0.})

exp = qtune.basic_dqd.TestExperiment(initial_voltages=initial_voltages,
                                     measurements=(default_detune_scan, default_reload_scan, sensing_dot_measurement,
                                                   ss2d_measurement, rfa_line_scan, rfb_line_scan),
                                     simulator_dict={id(default_detune_scan): detune_sim, id(default_reload_scan): singlet_reload_simulation,
                                                     id(sensing_dot_measurement): ss1d_sim, id(ss2d_measurement): ss2d_sim,
                                                     id(rfa_line_scan): rfa_trans_sim, id(rfb_line_scan): rfb_trans_sim})

In [ ]:
"""
Now comes the actual setup of the automated tuner. We will have to repeat the following steps for each group of 
parameters. The first group is the sensing dot.
"""
test_ss1d_evaluator = qtune.evaluator.SensingDot1D(exp,
                                                   measurements=(sensing_dot_measurement, ),
                                                   parameters=("position_SDB2", "current_signal", "optimal_signal"),
                                                   name='SensingDot1D')
test_ss2d_evaluator = qtune.evaluator.SensingDot2D(exp,
                                                   measurements=(ss2d_measurement,),
                                                   parameters=("position_SDB1", "position_SDB2"),
                                                   name='SensingDot2D'
                                                   )

In [ ]:
"""
For the sensing dot, two measurements are implemented: a one and a two dimensional scan of the Coulomb oscillations.
The measurements are in each case a list containing only one scan. The keyword argument "parameters" contains a list
of the parameter names. 
The one dimensional scan gives the current sensitivity (current_signal), the optimal sensitivity
(optimal_signal), which can be achieved by adjusting the voltage on the sweeping gate and the position of the highest 
sensitivity (position_SDB2).
The SensingDotTuner requires conditions to decide, when to use which scan. These conditions are specified in the target.
The minimal threshold (min_threshold) defines the condition, under which no voltage needs to be changed. In this case
a certain signal must be currently obtained. The cost threshold requires a certain sensitivity must be obtainable by 
changing only the gate voltage on the sweeping gate. If this condition is not met, the two dimensional scan is used.
This requires more measurement time but should yield a better sensitivity.
"""
min_threshold = pd.Series(data=[2e-3, -np.inf, -np.inf], index=["current_signal", "optimal_signal", "position_SDB2"])
cost_threshold = pd.Series(data=[-np.inf, 2e-3, -np.inf], index=["current_signal", "optimal_signal", "position_SDB2"])

In [ ]:
"""
The rescaling is implemented in the target as well. For each parameter can be given a number or NAN value.
The conditions are combined into the target by the make_target function. It expects pandas Series and returns the target
as pandas Dataframe.
"""
rescaling_factor = pd.Series(data=[1, 1, 1, 1],
                             index=["current_signal", "optimal_signal", "position_SDB2", "position_SDB1"])
ss_target = qtune.solver.make_target(minimum=min_threshold, cost_threshold=cost_threshold,
                                     rescaling_factor=rescaling_factor)

In [ ]:
"""
The ForwardingSolver is trivial. It receives values and only changes names to return the expected results. The tuner
implements the decision making on which scan to choose.
"""
fsolver = qtune.solver.ForwardingSolver(target=ss_target,
                                        values_to_position=pd.Series(data=["SDB1", "SDB2"],
                                                                     index=["position_SDB1", "position_SDB2"]),
                                        current_position=exp.read_gate_voltages())
ss_tuner = qtune.parameter_tuner.SensingDotTuner(cheap_evaluators=[test_ss1d_evaluator],
                                                 expensive_evaluators=[test_ss2d_evaluator],
                                                 gates=["SDB1", "SDB2"], solver=fsolver)

In [ ]:
"""
Lead Transition:
We control the chemical potential in the dots by the position of the charge diagram. This position is defined by 
positions of lead transitions. These transitions occur when the chemical potential inside a dot is exactly as large as
the chemical potential in a lead. By crossing over this transition an electron is pushed into the dot or pulled out.
We implement one Evaluator for each transition. In principle they could be implemented in a single evaluator as they are
always evaluated together.
"""

lta_evaluator = qtune.evaluator.LeadTransition(exp, shifting_gates=("RFB", ), sweeping_gates=("RFA", ),
                                               parameters=("position_RFA", ), measurements=(rfa_line_scan, ),
                                               name='LeadTransitionRFA')
ltb_evaluator = qtune.evaluator.LeadTransition(exp, shifting_gates=("RFA", ), sweeping_gates=("RFB", ),
                                               parameters=("position_RFB", ), measurements=(rfb_line_scan, ),
                                               name='LeadTransitionRFB')

In [ ]:
"""
Kalman filter:
We want to optimize the positions of the lead transitions by a gradient based optimization algorithm combined with the
Kalman filter. The KalmanGradient implements the Kalman filter. We set the number of gates (n_pos_dim) to 6 and the
number of parameters (n_values) to 1. The initial covariance matrix and the process noise are chosen heuristically. 
"""

rfa_kalman = qtune.kalman_gradient.KalmanGradient(n_pos_dim=6, n_values=1, initial_gradient=None,
                                                  initial_covariance_matrix=1e4 * np.eye(6),
                                                  process_noise=1e-2 * np.eye(6))


In [ ]:
"""
The GradientEstimator class administers the Gradient classes. In this case the KalmanGradientEstimator determines the 
maximal covariance. If any eigenvalue of the covariance matrix of the KalmanGradient is larger than maximum_covariance,
and update step of length epsilon is performed in the direction of the corresponding eigenvector. 
"""

voltages = exp.read_gate_voltages()[["BA", "BB", "N", "SA", "SB", "T"]]
voltages = voltages.sort_index()
rfa_gradient_estimator = qtune.gradient.KalmanGradientEstimator(rfa_kalman,
                                                                current_position=voltages,
                                                                current_value=1., maximum_covariance=1., epsilon=.1)

In [ ]:
"""
We proceed likewise for the second transition.
"""

rfb_kalman = qtune.kalman_gradient.KalmanGradient(n_pos_dim=6, n_values=1, initial_gradient=None,
                                                  initial_covariance_matrix=1e4 * np.eye(6),
                                                  process_noise=1e-2 * np.eye(6))
rfb_gradient_estimator = qtune.gradient.KalmanGradientEstimator(rfb_kalman,
                                                                current_position=voltages,
                                                                current_value=1., maximum_covariance=1., epsilon=.1)

In [ ]:
"""
The targets are simply defined by target values for the positions and tolerances. All elements which are given a desired
value NAN will be kept constant. This way, each solver respects also the parameter it does not tune directly.
"""
cd_target = qtune.solver.make_target(desired=pd.Series([0., 0., np.nan, np.nan],
                                                       ["position_RFA", "position_RFB", "parameter_tunnel_coupling",
                                                        "parameter_time_load"]),
                                     tolerance=pd.Series([.01, .01, np.nan, np.nan],
                                                         ["position_RFA", "position_RFB", "parameter_tunnel_coupling",
                                                          "parameter_time_load"]),
                                     rescaling_factor=pd.Series([1, 1, 1, 1],
                                                                ["position_RFA", "position_RFB",
                                                                 "parameter_tunnel_coupling", "parameter_time_load"]))
par_target = qtune.solver.make_target(
    desired=pd.Series([np.nan, np.nan, 2.20, 1.5],
                      ["position_RFA", "position_RFB", "parameter_tunnel_coupling", "parameter_time_load"]),
    tolerance=pd.Series([np.nan, np.nan, .05, .5],
                        ["position_RFA", "position_RFB", "parameter_tunnel_coupling", "parameter_time_load"]),
    rescaling_factor=pd.Series([1, 1, 100, 10],
                               ["position_RFA", "position_RFB",
                                "parameter_tunnel_coupling",
                                "parameter_time_load"]))

In [ ]:
"""
Now we consider the remaining parameters, which are the width of the inter dot transition and the singlet reload time.
Here we use again the simulators.
"""

test_line_evaluator = qtune.evaluator.InterDotTCByLineScan(experiment=exp, measurements=(default_detune_scan, ))
test_load_evaluator = qtune.evaluator.NewLoadTime(experiment=exp, measurements=(default_reload_scan,))

In [ ]:
"""
We want to tune them as well with the Kalman filter so that we proceed as we did for the lead transitions.
"""

line_kalman = qtune.kalman_gradient.KalmanGradient(n_pos_dim=6, n_values=1, initial_gradient=None,
                                                   initial_covariance_matrix=1e4 * np.eye(6))
line_gradient_estimator = qtune.gradient.KalmanGradientEstimator(line_kalman,
                                                                 current_position=voltages,
                                                                 current_value=1., maximum_covariance=1., epsilon=.1)
load_kalman = qtune.kalman_gradient.KalmanGradient(n_pos_dim=6, n_values=1, initial_gradient=None,
                                                   initial_covariance_matrix=1e4 * np.eye(6))
load_gradient_estimator = qtune.gradient.KalmanGradientEstimator(load_kalman,
                                                                 current_position=voltages,
                                                                 current_value=1., maximum_covariance=1., epsilon=.1)

In [ ]:
cd_solver = qtune.solver.NewtonSolver(target=cd_target,
                                      current_position=voltages,
                                      gradient_estimators=[rfa_gradient_estimator, rfb_gradient_estimator,
                                                           line_gradient_estimator, load_gradient_estimator])
par_solver = qtune.solver.NewtonSolver(target=par_target,
                                       current_position=voltages,
                                       gradient_estimators=[rfa_gradient_estimator, rfb_gradient_estimator,
                                                            line_gradient_estimator, load_gradient_estimator])

In [ ]:
"""
The SubsetTuner use only a specific set of gates to tune their parameters.
"""

cd_tuner = qtune.parameter_tuner.SubsetTuner(evaluators=[lta_evaluator, ltb_evaluator],
                                             gates=["BA", "BB", "N", "SA", "SB", "T"], solver=cd_solver)
par_tuner = qtune.parameter_tuner.SubsetTuner(evaluators=[test_line_evaluator, test_load_evaluator],
                                              gates=["BA", "BB", "N", "SA", "SB", "T"], solver=par_solver)

In [ ]:
"""
The AutoTuner stores its information in the folder given by storage_path. The tuning_hierarchy is given as list. It
describes the order, in which the parameters are tuned. In this case we tune first the sensing dot, then the position 
of the lead transitions and then the singlet reload time and the inter dot transition width. Any time a voltage is 
changed an Autotuner restarts with the first element of the hierarchy. In this case, it is the sensing dot.
"""

storage_path = r"Y:\GaAs\Autotune\Data\UsingPython\DryRunsRefactored"
auto_tuner = qtune.autotuner.Autotuner(exp, tuning_hierarchy=[ss_tuner, cd_tuner, par_tuner],
                                       hdf5_storage_path=storage_path)

In [ ]:
"""
We initialize an empty history to store the analyse the Autotuner during the runtime. You can use the GUI for convenient
tuning. Just press start in the emerging GUI window. You can press the button  labeled "Real Time Plot" for plotting 
voltages, parameters and gradient during the tuning. 
"""

hist = qtune.history.History(None)
w = gui.setup_default_gui(auto_tuner, hist)

In [ ]:
"""
To reload the data of a tuning run, you can simply initialize a History object with the path to the storage folder (by
default labeled with the time of the execution of the tuning).
"""
loading_path = storage_path + r"\2018_09_16_15_57_31_299648"
hist = qtune.history.History(loading_path)